In [36]:
import numpy as np
import pandas as pd
import igraph as ig

config = pd.read_csv('config.csv')
TMAX=config['TMAX'][0]
tick=config['data_tick'][0]
distribution=str(config['food_dis'][0])
times=int(TMAX/tick)

results_folder='Clustering_results/'
def find_index(array, element):
	result = np.where(array == element)
	if result[0].size > 0:
		return result[0][0]
	else:
		return -1

def find_distance(point1, point2):
    return np.linalg.norm(point1 - point2)

def load_centroids(times):
	centroid_list=[]
	for i in range(times):
		time=i*tick
		centroids=np.loadtxt(results_folder+f"centroids_{distribution}_{time}.csv",delimiter=',')
		centroid_list.append(centroids)

	return centroid_list


def find_ancestor(centroid,prev_centroids):
	distances=[]
	for ancestor in prev_centroids:
		distance=find_distance(centroid,ancestor)
		distances.append(distance)

	return np.argmin(distances)



In [37]:
centroid_list=load_centroids(times)
original_centers=centroid_list[0]
prev_centroids=original_centers

FileNotFoundError: Clustering_results/centroids_0_0.csv not found.

In [ ]:
# node_index={}
Graph = ig.Graph()

for time in range(times):
	centroids=centroid_list[time]
	
	ii=0

	for centroid in centroids:
		node=f"{time}_{ii}"
		Graph.add_vertex(node)
		if time>0:
			ancestor=find_ancestor(centroid,prev_centroids)
			ancestor_node=f"{time-1}_{ancestor}"
			Graph.add_edge(node,ancestor_node)
		ii+=1
	prev_centroids=centroids

	


In [ ]:
components = Graph.connected_components()
i=0
subgraphs=[]
layouts = []
for component in components:
	i+=1
	# print(component)
	subgraph = Graph.subgraph(component)
	subgraphs.append(subgraph)
	layout = subgraph.layout_reingold_tilford(mode="in", root=[0])
	layouts.append(layout)
	# ig.plot(subgraph, layout=layout, vertex_label=subgraph.vs["name"],target=f'subgraph_{i}.png')


In [38]:



sizes = [(np.array([coord for coord in layout]), np.ptp(layout, axis=0)) for layout in layouts]

translation = np.array([0, 0],dtype='float64')

translated_layouts = []

for layout, size in sizes:
    print(size)
    
    translation_width = 0*size[0] + 5
    translation += np.array([translation_width, 0], dtype='float64')
    translated_layout = layout + translation
    translated_layouts.append(translated_layout)

combined_layout = np.concatenate(translated_layouts)




colors = ['red', 'green', 'yellow', 'purple']  # Add more colors if needed
# TODO add HSV colors


# Assign a color to each vertex based on its subgraph
for i, subgraph in enumerate(subgraphs):
    for v in subgraph.vs:
        v['color'] = colors[i % len(colors)]

combined_graph = ig.Graph().disjoint_union(subgraphs)
vertex_labels = combined_graph.vs['name']
resolution = 100
plot_size = (16*resolution, 9*resolution)


scale= 0.3
plot = ig.plot(
    combined_graph, 
    layout=combined_layout,
    vertex_label=vertex_labels,
    vertex_label_size=10*scale,
    vertex_size=20*scale,
    bbox=plot_size,
    vertex_color=combined_graph.vs['color'],
    margin=150
    
	)

# Save the plot
print("xc")
plot.save('combined_graph.png')


[ 32.04166667 199.        ]
[0. 2.]
[0. 0.]
xc
